In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import cv2
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tqdm import tqdm
import os
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, TensorBoard, ModelCheckpoint
from sklearn.metrics import classification_report,confusion_matrix
import ipywidgets as widgets
import io
from PIL import Image
from IPython.display import display,clear_output
from warnings import filterwarnings

In [3]:
# define the path to the model
path_to_model = r'backend\model\trained\v1\effnet.h5'
# load the model
model = tf.keras.models.load_model(path_to_model)

In [4]:
def img_pred(upload):
    for name, file_info in uploader.value.items():
        img = Image.open(io.BytesIO(file_info['content']))
    opencvImage = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)
    img = cv2.resize(opencvImage,(150,150))
    img = img.reshape(1,150,150,3)
    p = model.predict(img)
    p = np.argmax(p,axis=1)[0]

    if p==0:
        p='Glioma Tumor'
    elif p==1:
        print('The model predicts that there is no tumor')
    elif p==2:
        p='Meningioma Tumor'
    else:
        p='Pituitary Tumor'

    if p!=1:
        print(f'The Model predicts that it is a {p}')

In [10]:
uploader = widgets.FileUpload()
display(uploader)

FileUpload(value=(), description='Upload')

In [11]:
button = widgets.Button(description='Predict')
out = widgets.Output()
def on_button_clicked(_):
    with out:
        clear_output()
        try:
            img_pred(uploader)
            
        except:
            print('No Image Uploaded/Invalid Image File')
button.on_click(on_button_clicked)
widgets.VBox([button,out])

In [12]:
# preprocessing the image
def preprocess_image(image_path):
    # Load the image
    img = cv2.imread(image_path)
    
    # Resize the image to the input size of the model
    img = cv2.resize(img, (150, 150))
    
    # Normalize the image
    img = img / 255.0
    
    # Expand dimensions to match the input shape of the model
    img = np.expand_dims(img, axis=0)
    
    return img

In [13]:
# prediction function
def predict_image(image_path):
    # Preprocess the image
    img = preprocess_image(image_path)
    
    # Make prediction
    predictions = model.predict(img)
    
    # Get the class with the highest probability
    predicted_class = np.argmax(predictions, axis=1)[0]
    
    return predicted_class

In [14]:
# mapping class indices to labels
class_labels = {0: 'Glioma Tumor', 1: 'No Tumor', 2: 'Meningioma Tumor', 3: 'Pituitary Tumor'}
# function to display the prediction result
def display_prediction(image_path):
    predicted_class = predict_image(image_path)
    label = class_labels[predicted_class]
    
    # Load and display the image
    img = Image.open(image_path)
    plt.imshow(img)
    plt.axis('off')
    plt.title(f'Predicted: {label}')
    plt.show()

# function to handle file upload and prediction
def handle_file_upload(change):
    if uploader.value:
        # Get the uploaded file
        uploaded_file = list(uploader.value.values())[0]
        # Save the uploaded file to a temporary location
        with open('temp_image.jpg', 'wb') as f:
            f.write(uploaded_file['content'])
        
        # Display the prediction
        display_prediction('temp_image.jpg')

In [15]:
# filterwarnings('ignore')
uploader = widgets.FileUpload(accept='.jpg,.jpeg,.png', multiple=False)
uploader.observe(handle_file_upload, names='value')
display(uploader)

FileUpload(value=(), accept='.jpg,.jpeg,.png', description='Upload')

AttributeError: 'tuple' object has no attribute 'values'